# Running an R Script with BOA

In [1]:
# This notebook uses hidden cells to import and display things so
# that it can be ran regullary to make sure the documentation
# is up to date and not broken (as opposed to a markdown file
# that would have an exmaple written once and get out of date
# as the code base changes)

# These hidden cells are only responsible for rerunning
# the documentation to ensure it is correct.
# the actual relavent part of the documentation
# are the non hidden parts
import pathlib
import re
import shutil

from IPython.display import Code

import boa

[WARNING 07-12 17:22:25] ax.service.utils.with_db_settings_base: Ax currently requires a sqlalchemy version below 2.0. This will be addressed in a future release. Disabling SQL storage in Ax for now, if you would like to use SQL storage please install Ax with mysql extras via `pip install ax-platform[mysql]`.


In [2]:
boa_dir = pathlib.Path().resolve().parent.parent
r_dir = boa_dir / "tests/scripts/other_langs/r_package_streamlined"
config_path = r_dir / "config.yaml"
wrapper_script = r_dir / "run_model.R"

config = boa.load_jsonlike(config_path)
ex_name = config["optimization_options"]["experiment"]["name"]
total_trials = config["optimization_options"]["trials"]

In [3]:
# Remove old runs to have a clean slate for this example
old_runs = pathlib.Path().resolve().glob(f"{ex_name}*")
for path in old_runs:
    shutil.rmtree(path)
old_runs = r_dir.glob(f"{ex_name}*")
for path in old_runs:
    shutil.rmtree(path)

In [4]:
Code(config_path)

optimization_options:
    objective_options:
        objectives:
            - name: metric
    experiment:
        name: "r_streamlined_run"
    trials: 15

parameters:
    x0:
        'bounds': [ 0, 1 ]
        'type': 'range'
        'value_type': 'float'
    x1:
        'bounds': [ 0, 1]
        'type': 'range'
        'value_type': 'float'
    x2:
        'bounds': [ 0, 1 ]
        'type': 'range'
        'value_type': 'float'
    x3:
        'bounds': [ 0, 1]
        'type': 'range'
        'value_type': 'float'
    x4:
        'bounds': [ 0, 1 ]
        'type': 'range'
        'value_type': 'float'
    x5:
        'bounds': [ 0, 1]
        'type': 'range'
        'value_type': 'float'

script_options:
    run_model: Rscript run_model.R


# options only needed by the model and not BOA
model_options:
    input_size: 15
    model_dir: .

In [5]:
Code(wrapper_script)

# load in any libraries and modules we need
library(jsonlite)
source("../r_utils/hartman6.R")

# This is where we read in from BOA the command line argument.
# If in your script, you use any other command line arguments,
# generally BOA's trial_dir should be the last command line arugment,
# so taking the last one should generally be safe.
args <- commandArgs(trailingOnly=TRUE)
trial_dir <- args[length(args)]

# this this trial_dir folder there are 2 files supplied by BOA,
# a parameters.json that has just the parameters, and a trial.json
# that includes the parameters and a lot more in case you need it.
# Most people will only need the parameters.json
param_path <- file.path(trial_dir, "parameters.json")
data <- read_json(path=param_path)

# The parameter keys config from whatever you  named them in your
# config file, which you are free to change.
x0 <- data$x0
x1 <- data$x1
x2 <- data$x2
x3 <- data$x3
x4 <- data$x4
x5 <- data$x5
X <- c(x0, x1, x2, x3, x4, x5)

# This is where we actually run our "model".
# Here we are using a synthetic function called hartman6
# But you could substitute it for your own model in
# a number of ways.
res <- hartman6(X)

# In this case, we directly ran the model, so we are getting back a number
# or nan, so we know if it succeeded or failed. If you are submitting a job
# to an HPC (a super computer) queue, this might work, or you might have to
# rely on another method. Other options could be relying on log file output
# or information from querying the queue itself,
# though those may be better as stand alone `Set Trial Status Scripts`
if (!is.na(res)) {

    # if it was a success, we don't even need to write out trial status,
    # it is assumed a success if we write out data and don't fail
    out_data <- list(
        metric=res
        # trial_status=unbox("COMPLETED")  #  this is optional if it succeeds
    )

} else {

    # If we fail, then we do need to include a trial status, and mark it as failed.
    out_data <- list(
        trial_status=unbox("FAILED")
    )
}

json_data <- toJSON(out_data, pretty = TRUE)
write(json_data, file.path(trial_dir, "output.json"))

In [6]:
output = !python -m boa -c {config_path}  # we capture ipython terminal output to python variable
o = "\n".join(ln for ln in output)  # it comes in as a list, we convert to string
o = o.replace(str(r_dir),  "[/path/to/your/dir/]")  # replace the actual dir with a stand in for privacy reasons
Code(o)

[WARNING 07-12 17:22:27] ax.service.utils.with_db_settings_base: Ax currently requires a sqlalchemy version below 2.0. This will be addressed in a future release. Disabling SQL storage in Ax for now, if you would like to use SQL storage please install Ax with mysql extras via `pip install ax-platform[mysql]`.
[INFO 07-12 17:22:28] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='x0', parameter_type=FLOAT, range=[0.0, 1.0]), RangeParameter(name='x1', parameter_type=FLOAT, range=[0.0, 1.0]), RangeParameter(name='x2', parameter_type=FLOAT, range=[0.0, 1.0]), RangeParameter(name='x3', parameter_type=FLOAT, range=[0.0, 1.0]), RangeParameter(name='x4', parameter_type=FLOAT, range=[0.0, 1.0]), RangeParameter(name='x5', parameter_type=FLOAT, range=[0.0, 1.0])], parameter_constraints=[]).
[INFO 07-12 17:22:28] ax.modelbridge.dispatch_utils: Using Models.GPEI since there are more ordered parameters than there are categories for the unordered categorical parameters.
[INFO 07-12 17:22:28] ax.modelbridge.dispatch_utils: Calculating the number of remaining initialization trials based on num_initialization_trials=None max_initialization_trials=None num_tunable_parameters=6 num_trials=None use_batch_trials=False
[INFO 07-12 17:22:28] ax.modelbridge.dispatch_utils: calculated num_initialization_trials=12
[INFO 07-12 17:22:28] ax.modelbridge.dispatch_utils: num_completed_initialization_trials=0 num_remaining_initialization_trials=12
[INFO 07-12 17:22:28] ax.modelbridge.dispatch_utils: Using Bayesian Optimization generation strategy: GenerationStrategy(name='Sobol+GPEI', steps=[Sobol for 12 trials, GPEI for subsequent trials]). Iterations after 12 will take longer to generate due to model-fitting.
[INFO 07-12 17:22:28] Scheduler: `Scheduler` requires experiment to have immutable search space and optimization config. Setting property immutable_search_space_and_opt_config to `True` on experiment.
[INFO 2023-07-12 17:22:28,222 MainProcess] boa: 

##############################################


BOA Experiment Run
Output Experiment Dir: [/path/to/your/dir/]/r_streamlined_run_20230712T172228
Start Time: 20230712T172228
Version: 0.8.7.dev0+g78bf53c.d20230712

##############################################

[INFO 07-12 17:22:28] Scheduler: Running trials [0]...
[INFO 07-12 17:22:29] Scheduler: Running trials [1]...
[INFO 07-12 17:22:31] Scheduler: Running trials [2]...
[INFO 07-12 17:22:32] Scheduler: Running trials [3]...
[INFO 07-12 17:22:34] Scheduler: Running trials [4]...
[INFO 07-12 17:22:35] Scheduler: Running trials [5]...
[INFO 07-12 17:22:37] Scheduler: Running trials [6]...
[INFO 07-12 17:22:38] Scheduler: Running trials [7]...
[INFO 07-12 17:22:39] Scheduler: Running trials [8]...
[INFO 07-12 17:22:40] Scheduler: Running trials [9]...
[INFO 07-12 17:22:42] Scheduler: Retrieved COMPLETED trials: 0 - 9.
[INFO 07-12 17:22:42] Scheduler: Fetching data for trials: 0 - 9.
[INFO 2023-07-12 17:22:42,175 MainProcess] boa: Saved JSON-serialized state of optimization to `[/path/to/your/dir/]/r_streamlined_run_20230712T172228/scheduler.json`.
Boa version: 0.8.7.dev0+g78bf53c.d20230712
[INFO 2023-07-12 17:22:42,190 MainProcess] boa: Trials so far: 10
Running trials: 
Will Produce next trials from generation step: Sobol
Best trial so far: {3: {'metric': -1.4904}}
[INFO 07-12 17:22:42] Scheduler: Running trials [10]...
[INFO 07-12 17:22:43] Scheduler: Running trials [11]...
[INFO 07-12 17:22:45] Scheduler: Running trials [12]...
[INFO 07-12 17:22:47] ax.modelbridge.torch: The observations are identical to the last set of observations used to fit the model. Skipping model fitting.
[INFO 07-12 17:22:48] Scheduler: Running trials [13]...
[INFO 07-12 17:22:49] ax.modelbridge.torch: The observations are identical to the last set of observations used to fit the model. Skipping model fitting.
[INFO 07-12 17:22:50] Scheduler: Running trials [14]...
[INFO 07-12 17:22:51] Scheduler: Retrieved COMPLETED trials: 10 - 

In [7]:
# These are just safety sanity checks for internal doc testing purposes

# Check that the number of trials in config is the number that got ran
trials_ran = int(
    # we look for [n rows x m columns], grab that row
    # then grab a regex of "n rows"
    re.search("\d+ rows", output.grep("\d+ rows x \d+ columns")[0]).group()
    .split()[0]  # split on the space and grab "n"
)  # cast to int

# Check that the experiment name in config is the one that actually got used and written to disk
# and the experiment actually got written to disk
ex_save_dir = [
    ln for ln in output.grep(ex_name) 
    if "Output Experiment Dir: " in ln
][0].split()[-1]
assert pathlib.Path(ex_save_dir).exists()